# **Libraries**
---

In [1]:
import pandas as pd
import nltk
import re
import os
import zipfile

# **KAGGLE**
---
Import *"Vietnamese Online News Dataset"* and "Vietnames Stopword" from Kaggle

In [3]:
# Set API Key and Username
os.environ["KAGGLE_USERNAME"] = "nguyenvanphiyen"
os.environ["KAGGLE_KEY"] = "c6aad4cde44af5d3f106340438161edf"

In [3]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!kaggle datasets download -d haitranquangofficial/vietnamese-online-news-dataset -p ./data --force

Dataset URL: https://www.kaggle.com/datasets/haitranquangofficial/vietnamese-online-news-dataset
License(s): CC0-1.0




  0%|          | 0.00/199M [00:00<?, ?B/s]
 39%|███▉      | 77.0M/199M [00:00<00:00, 728MB/s]
 74%|███████▍  | 147M/199M [00:00<00:00, 480MB/s] 
 99%|█████████▉| 197M/199M [00:00<00:00, 464MB/s]
100%|██████████| 199M/199M [00:00<00:00, 488MB/s]


In [7]:
# Vietnamese stopwords
!kaggle datasets download -d linhlpv/vietnamese-stopwords -p ./data --force

Dataset URL: https://www.kaggle.com/datasets/linhlpv/vietnamese-stopwords
License(s): unknown




  0%|          | 0.00/6.73k [00:00<?, ?B/s]
100%|██████████| 6.73k/6.73k [00:00<00:00, 3.73MB/s]


In [8]:
import zipfile

folder = "./data"

for file in os.listdir(folder):
    if file.endswith(".zip"):
        zip_path = os.path.join(folder, file)

        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(folder) 

# **READ DATASET**
---

In [10]:
news_dataset = pd.read_json(folder + '/news_dataset.json')
news_dataset.head()

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170


In [11]:
# Vietnamese Stopwords
with open(folder + '/vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()

stopwords = [w.strip() for w in stopwords]
stopwords

['a lô',
 'a ha',
 'ai',
 'ai ai',
 'ai nấy',
 'ai đó',
 'alô',
 'amen',
 'anh',
 'anh ấy',
 'ba',
 'ba ba',
 'ba bản',
 'ba cùng',
 'ba họ',
 'ba ngày',
 'ba ngôi',
 'ba tăng',
 'bao giờ',
 'bao lâu',
 'bao nhiêu',
 'bao nả',
 'bay biến',
 'biết',
 'biết bao',
 'biết bao nhiêu',
 'biết chắc',
 'biết chừng nào',
 'biết mình',
 'biết mấy',
 'biết thế',
 'biết trước',
 'biết việc',
 'biết đâu',
 'biết đâu chừng',
 'biết đâu đấy',
 'biết được',
 'buổi',
 'buổi làm',
 'buổi mới',
 'buổi ngày',
 'buổi sớm',
 'bà',
 'bà ấy',
 'bài',
 'bài bác',
 'bài bỏ',
 'bài cái',
 'bác',
 'bán',
 'bán cấp',
 'bán dạ',
 'bán thế',
 'bây bẩy',
 'bây chừ',
 'bây giờ',
 'bây nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên bị',
 'bên có',
 'bên cạnh',
 'bông',
 'bước',
 'bước khỏi',
 'bước tới',
 'bước đi',
 'bạn',
 'bản',
 'bản bộ',
 'bản riêng',
 'bản thân',
 'bản ý',
 'bất chợt',
 'bất cứ',
 'bất giác',
 'bất kì',
 'bất kể',
 'bất kỳ',
 'bất luận',
 'bất ngờ',
 'bất nhược',
 'bất quá',
 'bất quá chỉ',
 'bất thình l

In [12]:
token_stopwords = ['_'.join(stop_word.split()) for stop_word in stopwords]
token_stopwords

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

# **Merge Author - Title - Contents**
---
Merge three columns: Author, Title, Contents

In [13]:
news_dataset['article'] = news_dataset['author'].fillna('') + ' ' + news_dataset['title'].fillna('') + ' ' + news_dataset['content'].fillna(' ')

display(news_dataset['article'].head())

0     Tên cướp tiệm vàng tại Huế là đại uý công an,...
1    (Nguồn: Sina) Bỏ qua mạng 5G, Nga tiến thẳng t...
2    Hồ Sỹ Anh Địa phương nào đứng đầu cả nước tổng...
3    Ngọc Ánh Người chết trong mưa lũ 'nghìn năm có...
4    HẢI YẾN - MINH LÝ Hải Phòng: Hình ảnh xe "điên...
Name: article, dtype: object

# **Lowercase**
---


In [14]:
news_dataset['article']=news_dataset['article'].str.lower()
news_dataset['article'].head()

0     tên cướp tiệm vàng tại huế là đại uý công an,...
1    (nguồn: sina) bỏ qua mạng 5g, nga tiến thẳng t...
2    hồ sỹ anh địa phương nào đứng đầu cả nước tổng...
3    ngọc ánh người chết trong mưa lũ 'nghìn năm có...
4    hải yến - minh lý hải phòng: hình ảnh xe "điên...
Name: article, dtype: object

# **Remove Punctuation**
---

In [15]:
news_dataset['article'] = news_dataset['article'].str.replace(r'[^\w\s]', '', regex=True)
news_dataset['article'].head()

0     tên cướp tiệm vàng tại huế là đại uý công an ...
1    nguồn sina bỏ qua mạng 5g nga tiến thẳng từ 4g...
2    hồ sỹ anh địa phương nào đứng đầu cả nước tổng...
3    ngọc ánh người chết trong mưa lũ nghìn năm có ...
4    hải yến  minh lý hải phòng hình ảnh xe điên gâ...
Name: article, dtype: object

# **Tokenization**
---
For Vietnamese text data, I use **underthesea** library to preprocess.
> **underthesea** is a suite of open source Python modules, data sets and tutorials supporting research and development in Vietnamese Natural Language Processing


```
$ pip install underthesea
```

```
from underthesea import sent_tokenize
from underthesea import pos_tag
from underthesea import chunk
from underthesea import dependency_parse
from underthesea import ner
from underthesea import classify
```
But maybe you will have trouble when installing and using underthesea. Many people have faced the same problem: ModuleNotFoundError: No module named 'maturin'.
Note that underthesea only supports Python <= 3.11.

In this case, you can use a different Vietnamese NLP library such as **VnCoreNLP** or **PyVi** instead.


In [16]:
!pip install pyvi

Defaulting to user installation because normal site-packages is not writeable


In [17]:
from pyvi import ViTokenizer, ViPosTagger

In [18]:
news_dataset['article_tokens'] = news_dataset['article'].apply(lambda x: ViTokenizer.tokenize(x))


In [19]:
news_dataset['article_tokens'].head()

0    tên cướp tiệm vàng tại huế là đại_uý công_an c...
1    nguồn sina bỏ_qua mạng 5g nga tiến thẳng từ 4g...
2    hồ_sỹ anh địa_phương nào đứng đầu cả nước tổng...
3    ngọc ánh người chết trong mưa_lũ nghìn năm có ...
4    hải_yến minh lý hải phòng hình_ảnh xe điên gây...
Name: article_tokens, dtype: object

# **Remove Stopwords**
* * *
Remove stopwords from the tokenized articles.

In [20]:
news_dataset['article_tokens_list'] = news_dataset['article_tokens'].apply(lambda x: x.split())

news_dataset['article_tokens_nostop_list'] = news_dataset['article_tokens_list'].apply(
    lambda tokens: [tok for tok in tokens if tok not in token_stopwords]
)


In [21]:
news_dataset['article_tokens_nostop_list'].head()

0    [cướp, tiệm, vàng, huế, đại_uý, công_an, công_...
1    [sina, bỏ_qua, mạng, 5g, nga, tiến, thẳng, 4g,...
2    [hồ_sỹ, địa_phương, đứng, đầu, tổng_điểm, 3, m...
3    [ngọc, ánh, chết, mưa_lũ, nghìn, mỹ, 28, thống...
4    [hải_yến, minh, lý, hải, phòng, hình_ảnh, xe, ...
Name: article_tokens_nostop_list, dtype: object

# **POS Tagging**
---
POS tagging: Labeling each word in a text as a noun, verb, adjective, or other part of speech.

In [23]:
news_dataset['pos_tagged_pairs'] = news_dataset['article_tokens_nostop_list'].apply(
    lambda x: list(zip(*ViPosTagger.postagging_tokens(x)))
)


In [25]:
news_dataset['pos_tagged_pairs']

0         [(cướp, V), (tiệm, N), (vàng, N), (huế, N), (đ...
1         [(sina, N), (bỏ_qua, V), (mạng, N), (5g, M), (...
2         [(hồ_sỹ, N), (địa_phương, N), (đứng, V), (đầu,...
3         [(ngọc, N), (ánh, Nc), (chết, V), (mưa_lũ, N),...
4         [(hải_yến, V), (minh, N), (lý, N), (hải, V), (...
                                ...                        
184534    [(quốc, N), (thắng, V), (nghi, N), (can, N), (...
184535    [(thảo, N), (mi, P), (tất, V), (thành, V), (ca...
184536    [(tâm, N), (nga, V), (tuyên_bố, N), (giải_phón...
184537    [(lâm, V), (thỏa_hlv, N), (malaysia, N), (dọa,...
184538    [(lang, V), (minh, N), (giáo_dục, V), (thành, ...
Name: pos_tagged_pairs, Length: 184539, dtype: object